In [5]:
# Импортируем необходимые библиотеки:
# pandas для манипуляции данными,
# scikit-surprise для функциональности рекомендательных систем, такой как чтение датасетов и применение алгоритмов.
import pandas as pd
from surprise import Dataset, Reader, SVD

# Загружаем датасет с рейтингами фильмов.
ratings = pd.read_csv("ratings.csv")

# Создаем таблицу сводки, чтобы увидеть рейтинги, которые пользователи ставят каждому фильму.
# Индекс: userId, Колонки: movieId, Значения: рейтинг.
user_movie_rating = ratings.pivot_table(index='userId', columns='movieId', values='rating')
# Показываем первые 100 строк для быстрого просмотра.
user_movie_rating.head(100)  

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98,4.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Определяем объект Reader, указывая шкалу рейтинга.
reader = Reader(rating_scale=(0.5, 5))  # Рейтинги фильмов варьируются от 0.5 до 5.

# Загружаем данные из DataFrame.
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

# Строим полный тренировочный набор данных.
trainset = data.build_full_trainset()

# Создаем экземпляр алгоритма SVD.
# Если алгоритму передан аргумент biased=true (по умолчанию true),
# Scikit-compose реализует Знаменитый алгоритм SVD, популяризированный Саймоном Фанком во время премии Netflix.
# Если biased=false, то будет использован Probabilistic Matrix Factorization, т.е. SVD без весов.
algorithm = SVD(biased=True)
 # Обучаем модель на тренировочном наборе.
algorithm.fit(trainset) 

# Подготавливаем пустой список для тестового набора данных.
testset = []

# Проходим по всем пользователям и фильмам для создания тестового набора.
for uid in trainset.all_users():
    for iid in trainset.all_items():
        # Проверяем отсутствие пары (uid, iid) в тренировочном наборе.
        if not trainset.ur.get(uid) or (iid not in map(lambda x:x[0], trainset.ur[uid])):
            # Добавляем в тестовый набор с None в качестве фактического рейтинга.
            testset.append((trainset.to_raw_uid(uid), trainset.to_raw_iid(iid), None))

# Получаем предсказания для отсутствующих значений.
predictions = algorithm.test(testset)

# Подготавливаем список для хранения предсказанных рейтингов.
pred_ratings = []
for pred in predictions:
    pred_ratings.append([pred.uid, pred.iid, pred.est])

In [ ]:
# Преобразуем список предсказаний в DataFrame.
pred_df = pd.DataFrame(pred_ratings, columns=['userId', 'movieId', 'rating'])

# Объединяем предсказанные рейтинги с исходными рейтингами.
complete_ratings = pd.concat([ratings[['userId', 'movieId', 'rating']], pred_df])

# Создаем итоговую таблицу сводки с полными данными, включающую как фактические, так и предсказанные рейтинги.
complete_user_movie_rating = complete_ratings.pivot_table(index='userId', columns='movieId', values='rating')

# Сортируем столбцы итоговой сводной таблицы согласно исходной таблице сводки рейтингов.
complete_user_movie_rating = complete_user_movie_rating[user_movie_rating.columns]

# Показываем первые 100 строк итоговой таблицы для быстрого просмотра.
complete_user_movie_rating.head(100)

In [ ]:
# Экспортируем итоговую таблицу в CSV файл.
complete_user_movie_rating.to_csv("predicted_ratings.csv")